<a href="https://colab.research.google.com/github/smnststn/Fruits_vegetables/blob/main/Final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub
import os
from PIL import Image
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Input,
    Dropout
)

from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.applications import MobileNetV3Large
from keras.layers import GlobalAveragePooling2D
from keras.models import Model

c:\Users\CO2\Desktop\Final project\Fruits_vegetables\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [55]:
def plot_history(history):
  """
  Funkce na vykreslení dat z objectu history -> vrací ho metoda fit keras modelu
  """
  # plot the Loss curve(s)
  plt.figure(figsize=[8,6])

  plt.plot(history.history['loss'],'r',linewidth=3.0)
  plt.plot(history.history['val_loss'],'b',linewidth=3.0)

  plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
  plt.xlabel('Epochs ',fontsize=16)
  plt.ylabel('Loss',fontsize=16)
  plt.title('Loss Curves',fontsize=16)
  plt.show()

  # plot the Accuracy curve(s)
  plt.figure(figsize=[8,6])

  plt.plot(history.history['accuracy'], 'r', linewidth=3.0)
  plt.plot(history.history['val_accuracy'], 'b',linewidth=3.0)

  plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
  plt.xlabel('Epochs ',fontsize=16)
  plt.ylabel('Accuracy',fontsize=16)
  plt.title('Accuracy Curves',fontsize=16)
  plt.show()

In [5]:
path = "C:\\Users\\CO2\\Desktop\\Final project\\Fruits_vegetables\\Data"
if os.path.exists(path):
    print(f"Složka '{path}' existuje.")
else:
    print(f"Složka '{path}' neexistuje.")

Složka 'C:\Users\CO2\Desktop\Final project\Fruits_vegetables\Data' existuje.


In [57]:
pass

#import kagglehub

    # Download latest version
    #path = kagglehub.dataset_download("kritikseth/fruit-and-vegetable-image-recognition")

    #print("Path to dataset files:", path)

**KOMENTÁR Šimon:**

* pridal som funkciu load images, v ktorej som využil kód, na ktorom sme
pracovali v prvý deň. Túto funkciu použijeme pre každý folder (train/test/validation)
* trochu som premenoval niektoré premenné do angličtiny a v komentároch vysvetlil, ten komentár tám môžeme nechať aj pre prehľadnosť k obhajobe
* podarilo sa mi nájsť zjednodušenie prevodu grayscale/RGBA na RGB
* snažil som sa o prehľadnosť a úspornosť kódu

**Otázka na zváženie:**
- Chceme mať radšej premenné a názvy funkcií radšej v češtine, napr. namiesto *load_images --> načíst_data*

In [6]:
train_dir = os.path.join(path, 'train')
validation_dir = os.path.join(path, 'validation')
test_dir = os.path.join(path, 'test')

In [9]:
def load_images(path, folder, image_width = 224, image_height = 224):
  """
  Function returns  X=images, y=labels,
  encoding=labels for each category
  """

  X = []
  y = []
  encoding = {}

  '''
  path = path to dataset
  folder = train, validation or test
  category = vegetable or fruit
  filename = name of image

  path to image = path + folder + category + filename
             e.g. path / train / beetroot / Image_1.jpg
  '''

  folder_path = os.path.join(path, folder)

  # Loop for each category (fruit/vegetable)
  for index, category in enumerate(os.listdir(folder_path)): # (folder_path)[0:7] --> only first 7 categories / to be removed

    encoding[index] = category
    category_path = os.path.join(folder_path, category)
    image_list = []

    # Loop for each file
    for filename in os.listdir(category_path):

      # Including all formats
      if filename.endswith((".jpg",".png",".jpeg")):
        img_path = os.path.join(category_path, filename)
        img = Image.open(img_path)

        # Converts to RGB (in case image mode is RGBA = with transparent background)
        img.convert('RGB')

        # Resizes to requested width/height (default 224x224)
        img_resized = img.resize((image_width, image_height))
        img_array = np.array(img_resized)

        if img_array.ndim == 2:  # If grayscale, convert to RGB
            img_array = np.stack((img_array,) * 3, axis=-1) # Convert to 3 channels if needed
            # Check if the image has 4 channels (RGBA) and convert if necessary
        elif img_array.shape[2] == 4:  # If RGBA, convert to RGB
            img_array = img_array[:,:,:3] # Removing the alpha channel by selecting only the first 3 (RGB) channels.
        image_list.append(img_array)

    X += image_list
    y += [index] * len(image_list)

  return X, y, encoding

In [10]:
X_test, y_test, encoding_test = load_images(path, 'test')
X_train, y_train, encoding_train = load_images(path, 'train')
X_val, y_val, encoding_val = load_images(path, 'validation')

c:\Users\CO2\Desktop\Final project\Fruits_vegetables\.venv\Lib\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [77]:
X_test[20]

array([[[241, 142,  90],
        [238, 152, 108],
        [230, 147, 102],
        ...,
        [115, 174,  76],
        [125, 184,  97],
        [154, 187, 127]],

       [[234, 125,  61],
        [231, 128,  67],
        [232, 143,  95],
        ...,
        [143, 188, 110],
        [116, 173,  79],
        [148, 171,  98]],

       [[236, 139,  85],
        [208, 112,  52],
        [161,  80,  25],
        ...,
        [115, 173,  72],
        [113, 167,  59],
        [130, 172,  54]],

       ...,

       [[ 64,  37,  15],
        [ 48,  24,   6],
        [ 54,  32,   9],
        ...,
        [138, 117,  86],
        [ 86,  58,  32],
        [149, 111,  87]],

       [[156, 127,  73],
        [113,  75,  39],
        [105,  70,  42],
        ...,
        [ 94,  75,  66],
        [ 72,  43,  32],
        [145, 102,  77]],

       [[177, 165, 118],
        [160, 139,  89],
        [142, 107,  69],
        ...,
        [ 73,  56,  51],
        [ 62,  38,  30],
        [144, 100,  77]]

In [78]:
encoding_test[y_test[0]]

'apple'

**Kontrola, či indexy a priradené kategórie (názvy ovocia/zeleniny) sedia**

In [33]:
# Check if indexes are identical across all encodings
if set(encoding_test.keys()) != set(encoding_train.keys()) or set(encoding_test.keys()) != set(encoding_val.keys()):
  print("(!) Inconsistent indexes across encodings")
else: print("Index are consistent")

# Check if labels are identical across all encodings.
for key in encoding_test.keys():
  if encoding_test[key] != encoding_train[key] or encoding_test[key] != encoding_val[key]:
    print(f"(!) Inconsistent label for index {key}: \n test: {encoding_test[key]},\n train: {encoding_train[key]},\n val: {encoding_val[key]}")
  else: print(f"Labels for index {key} are consistent")

Index are consistent
Labels for index 0 are consistent
Labels for index 1 are consistent
Labels for index 2 are consistent
Labels for index 3 are consistent
Labels for index 4 are consistent
Labels for index 5 are consistent
Labels for index 6 are consistent


In [11]:
# převést list s numpy arrayi pro každou image na numpy array (tenzor)
X_train = np.stack(X_train)
X_test = np.stack(X_test)
X_val = np.stack(X_val)
# převést list y na numpy array
y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)

In [12]:
# transfer learning viz. níže -> zvolili jsme Mobilnetv3 jako základ
# ten vyžaduje hodnoty pixelů v rozmezí od 0-255 -> neškálovat!
X_train_not_scaled = X_train.copy()
X_test_not_scaled = X_test.copy()
X_val_not_scaled = X_val.copy()

In [13]:
# škálujeme pro náš vlastní model
X_train = X_train / 255
X_test = X_test / 255
X_val = X_val / 255

In [14]:
# one hot encoding y -> musí sedět rozměr s výstupem softmax vrstvy
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

In [15]:
early = EarlyStopping(
    # sleduj přesnost u validačních dat
    monitor="val_accuracy",
    # pokud se nezlepší alespoň o 0.3%
    min_delta=0.003,
    # za posledních 10 epoch
    patience=10,
    # tak to vypni, a načti váhy z epochy, kdy byla nejvyšší přesnost na validačních datech
    restore_best_weights=True,
)

save = ModelCheckpoint(
    # kam uložit model
    filepath="best_model.keras",
    # sleduj přesnost u validačních dat
    monitor="val_accuracy",
    # ulož vždy jen jeden soubor s nejlepší hodnotou metriky
    save_best_only=True,
    # ulož do jednoho souboru architekturu i váhy
    save_weights_only=False,
    # po každé epoše
    save_freq="epoch"
)

In [ ]:
# vytvoříme vlastní model - inspirace VGG16 architektura
model = Sequential()
# vstup musí sedět na X
model.add(Input(shape=X_train.shape[1:]))
# blok konvoluce s poolingem
model.add(Conv2D(16, (3, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3, 3)))
model.add(MaxPooling2D())
# převést na vektor
model.add(Flatten())
# regularizace proti overfittingu


# výstupní vrstva -> multiclass single label -> softmax
model.add(Dense(36, activation="softmax"))

# kompilace
model.compile(loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=1000, validation_data=(X_test, y_test), batch_size=32, callbacks=[early, save])

In [ ]:
plot_history(history)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
# Predikce na testovacích datech
y_pred = model.predict(X_test)

# Převést predikce na labely (indexy kategorií)
y_pred = np.argmax(y_pred, axis=1)

# Převést one-hot encoded y_test na labely (indexy kategorií)
y_test_labels = np.argmax(y_test, axis=1)

# Tisk classification reportu
print(classification_report(y_test_labels, y_pred, target_names=np.array(list(encoding_test.values()))))

In [ ]:
# Redefine class labels from the encoding dictionary
class_labels = list(encoding_test.values())

# Assuming you already loaded your test data into X_test_not_scaled and y_test
# Get the true labels and predicted labels
true_labels = np.argmax(y_test, axis=1)  # Convert one-hot to class indices
pred_labels = model.predict(X_test)
pred_labels = np.argmax(pred_labels, axis=1) # Convert predictions to class indices

# Generate the confusion matrix
cm = confusion_matrix(true_labels, pred_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)

# Plot the confusion matrix
plt.figure(figsize=(12, 12))
disp.plot(cmap=plt.cm.Blues, values_format='d', ax=plt.gca())
plt.title("Confusion Matrix")
plt.xticks(rotation=90)
plt.show()